<a href="https://colab.research.google.com/github/tpeterz/happy-flix/blob/main/Recommendation_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation Tool Creation
# Integrating with Gradio interface
---
This Colab Notebook reads in the csv file that contains all movies that are ready for this recommendation function to analyze
# This will:



*   Read the file and parse the emotion and genres
*   Extract emotion and genres from the file content
*   Filter movies based on the predicted emotion
*   If no movies match the emotion, return a message
*   Further filter movies based on the selected genres
*   Adjust the genre name to match the DataFrame column format
*   If no movies match the genres, return a message
*   Rank movies based on the number of criteria they match and emotion score
*   Add the emotion score to the match count
*   Construct the poster URL
*   Add the movie to the ranked list
*   Sort the ranked list by match count (descending)
*   Select the top 3 movies
*   Format the recommendations for display

---
#### This would then output movie recommendations in a format like this example:

---

> Movie ID: 26156, Title: Hiding Out, Poster URL: https://image.tmdb.org/t/p/w300/jtGLfT8ckPmQgVhB3qhb3p0mnv8.jpg, Emotion: Fear, Genres: comedy, adventure, action
---
> Movie ID: 9448, Title: The Big Hit, Poster URL: https://image.tmdb.org/t/p/w300/aYDC6E9wi6vUmqnJtHsYJfMhFo2.jpg, Emotion: Fear, Genres: comedy, adventure, action
---
> Movie ID: 10136, Title: The Golden Child, Poster URL: https://image.tmdb.org/t/p/w300/9gPphtcGmna90U9CEsWlUVaSkAw.jpg, Emotion: Fear, Genres: comedy, adventure, action



---









In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Load the DataFrame
movie_predictions_df = pd.read_csv('recommendation_ready.csv')

movie_predictions_df.head()

,Unnamed: 0,genre_ids,movie_id,popularity,release_date,title,clean_reviews,type_Action,type_Adventure,type_Animation,...,label,score,anger,disgust,fear,joy,neutral,sadness,surprise,poster_path
0,0,"[878, 12]",693134,4534.956,2024-02-27,Dune: Part Two,dune part two surpasses even highest expectati...,0,1,0,...,joy,0.934813,0.002814,0.001023,0.005133,0.934813,0.037235,0.002196,0.016785,/1pdfLvkbY9ohJlCjQH2CZjjYVvJ.jpg
1,1,"[16, 28, 12, 35, 10751]",1011985,3214.314,2024-03-02,Kung Fu Panda 4,kung fu panda isnt best kung fu panda film eve...,1,1,1,...,sadness,0.598701,0.002053,0.006893,0.003863,0.068874,0.242003,0.598701,0.077614,/f7QBvIzoWSJw3jWPGnZBc5vwQ0l.jpg
2,2,"[28, 878, 12, 14]",823464,2297.340,2024-03-27,Godzilla x Kong: The New Empire,godzilla x kong new empire version previous in...,1,1,0,...,neutral,0.559732,0.173443,0.093400,0.059591,0.041290,0.559732,0.035302,0.037241,/tMefBSflR6PGQLv7WvFPpKLZkyk.jpg
3,3,"[28, 53]",359410,1134.726,2024-03-08,Road House,poor scenario story add ending also convincing...,1,0,0,...,joy,0.893423,0.002555,0.003341,0.002124,0.893423,0.077729,0.005483,0.015345,/bXi6IQiQDHD00JFio5ZSZOeRSBh.jpg
4,4,"[28, 14]",634492,921.140,2024-02-14,Madame Web,start heavily pregnant woman deep peruvian jun...,1,0,0,...,neutral,0.433036,0.070495,0.246801,0.108896,0.004469,0.433036,0.016794,0.119508,/rULWuutDcN5NvtiZi4FRPzRYWSh.jpg


In [ ]:
movie_predictions_df.rename(columns={"A": "a", "B": "c"})

In [ ]:
movie_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6123 entries, 0 to 6122
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            6123 non-null   int64  
 1   genre_ids             6123 non-null   object 
 2   movie_id              6123 non-null   int64  
 3   popularity            6123 non-null   float64
 4   release_date          6123 non-null   object 
 5   title                 6123 non-null   object 
 6   clean_reviews         6122 non-null   object 
 7   type_Action           6123 non-null   int64  
 8   type_Adventure        6123 non-null   int64  
 9   type_Animation        6123 non-null   int64  
 10  type_Comedy           6123 non-null   int64  
 11  type_Crime            6123 non-null   int64  
 12  type_Documentary      6123 non-null   int64  
 13  type_Drama            6123 non-null   int64  
 14  type_Family           6123 non-null   int64  
 15  type_Fantasy         

In [ ]:
with open('movie_output.txt', 'r') as f:
    lines = f.readlines()

print(lines)

In [ ]:
import pandas as pd

def recommend_movies(file):
    # Read the file and parse the emotion and genres
    with open(file.name, 'r') as f:
        lines = f.readlines()

    # Extract emotion and genres from the file content
    emotion = lines[0].split(': ')[1].strip().lower() # Convert emotion to lowercase
    genres = [genre.strip().lower() for genre in lines[1].split(': ')[1].strip().split(', ')] # Convert each genre to lowercase

    # Filter movies based on the predicted emotion
    emotion_movies = movie_predictions_df[movie_predictions_df['label'].str.lower() == emotion]

    # This sets the file path to be generated for each unique movie to be movies with the poster_path and the equation that creates it
    base_url = "https://image.tmdb.org/t/p/"
    file_size = "w300"


    # If no movies match the emotion, return a message
    if emotion_movies.empty:
        return [{"message": "No movies match your emotion preference."}]

    # Further filter movies based on the selected genres
    genre_movies = emotion_movies.copy()
    for genre in genres:
        # Adjust the genre name to match the DataFrame column format
        genre_column = f"type_{genre.capitalize().replace(' ', '_')}"
        genre_movies = genre_movies[genre_movies[genre_column] == 1]

    # If no movies match the genres, return a message
    if genre_movies.empty:
        return [{"message": "No movies match your genre preferences."}]

    # Rank movies based on the number of criteria they match and emotion score
    ranked_movies = []
    for index, row in genre_movies.iterrows():
        # Add the emotion score to the match count
        match_count = row[emotion]

        # Construct the poster URL
        poster_url = f"{base_url}{file_size}{row['poster_path']}"
        # Add the movie to the ranked list
        ranked_movies.append((row['movie_id'], row['title'], poster_url, match_count, row['clean_overviews']))

    # Sort the ranked list by match count (descending)
    ranked_movies.sort(key=lambda x: x[3], reverse=True)

    # Select the top 3 movies
    top_movies = ranked_movies[:3]

    # Format the recommendations for display
    recommendations = [{
        "id": movie[0],
        "title": movie[1],
        "poster_url": movie[2],
        "overview": movie[4],
        "emotion": emotion.capitalize(),
        "genres": ", ".join(genres)
    } for movie in top_movies]
    return recommendations

# Example usage
# This is only going to pull in the same movie_output.txt file that the original Gradio/OpenAI
# Interface created

# In theory, the openai model with the gradio interface would take in the aduio input, analyze the sentiment(emoitons) and genres (up to 3)
# The process would be fed through the model to produce recommendations, in the format displayed in the output of this cell
# The same or very similar function will be used with the other file in our repository to (in theory) generate recommendations.

with open('movie_output.txt', 'r') as file:
    recommendations = recommend_movies(file)

# Print the recommendations
for rec in recommendations:
    print(f"Movie ID: {rec['id']}, Title: {rec['title']}, Poster URL: {rec['poster_url']}, Emotion: {rec['emotion']}, Genres: {rec['genres']}")

Movie ID: 26156, Title: Hiding Out, Poster URL: https://image.tmdb.org/t/p/w300/jtGLfT8ckPmQgVhB3qhb3p0mnv8.jpg, Emotion: Fear, Genres: comedy, adventure, action
Movie ID: 9448, Title: The Big Hit, Poster URL: https://image.tmdb.org/t/p/w300/aYDC6E9wi6vUmqnJtHsYJfMhFo2.jpg, Emotion: Fear, Genres: comedy, adventure, action
Movie ID: 10136, Title: The Golden Child, Poster URL: https://image.tmdb.org/t/p/w300/9gPphtcGmna90U9CEsWlUVaSkAw.jpg, Emotion: Fear, Genres: comedy, adventure, action
